<a href="https://colab.research.google.com/github/XingxinHE/FinalThesis_DL-GA/blob/master/Compute_loss_between_mesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install torch torchvision
import sys
import torch
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-zekcke7l
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-zekcke7l
  Running command git checkout -q 701bbef4f34001c47ec2631ffd20d30d8fc241ea
ERROR: Operation cancelled by user


In [29]:
import os
import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
import numpy as np
from tqdm.notebook import tqdm
%matplotlib notebook 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [30]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
os.chdir("/content/gdrive/My Drive/Final Thesis/Textile/mesh/test_mesh")

## Function: load obj -> point cloud

In [41]:
def obj_to_cloud(filename):
  this_obj = os.path.join(filename)

  # load_obj
  verts, faces, aux = load_obj(this_obj)

  # transfer to tensor
  faces_idx = faces.verts_idx.to(device)
  verts = verts.to(device)

  # construct to PyTorch3d data structure
  this_mesh = Meshes(verts=[verts], faces=[faces_idx])

  # sample pt from mesh
  this_point_cloud = sample_points_from_meshes(this_mesh, 5000)

  x, y, z = this_point_cloud.clone().detach().cpu().squeeze().unbind(1)    
  fig = plt.figure(figsize=(15, 15))
  ax = Axes3D(fig)
  ax.scatter3D(x, z, -y)
  ax.set_xlabel('x')
  ax.set_ylabel('z')
  ax.set_zlabel('y')
  ax.set_title('fig_{}.png'.format(filename+' non-normalized version'))
  ax.view_init(190, 30)
  plt.savefig('fig_{}.png'.format(filename+' non-normalized version'))
  plt.show()

  return this_point_cloud

In [42]:
def compute_loss(filename1, filename2):


  sample_trg = obj_to_cloud(filename1)
  sample_src = obj_to_cloud(filename2)
  
  loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    
  # and (b) the edge length of the predicted mesh
  #loss_edge = mesh_edge_loss(sample_src)
    
  # mesh normal consistency
  #loss_normal = mesh_normal_consistency(sample_src)
    
  # mesh laplacian smoothing
  #loss_laplacian = mesh_laplacian_smoothing(sample_src, method="uniform")
    
  # Weighted sum of the losses
  #loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian
  message = "The chamfer distance between "+filename1+" and "+filename2+" is "+str(loss_chamfer.item())

  return message

In [34]:
m1 = compute_loss('test_original.obj', 'test_only_move.obj')
m2 = compute_loss('test_original.obj', 'test_0.5.obj')
m3 = compute_loss('test_original.obj', 'test_2.obj')
m4 = compute_loss('test_original.obj', 'test_ro1.obj')
m5 = compute_loss('test_original.obj', 'test_ro2_move.obj')
m6 = compute_loss('test_original.obj', 'test_sc_mv_ro.obj')
m7 = compute_loss('test_original.obj', '29_0.obj')

/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_original.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_only_move.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_sc_mv_ro.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
print(m1)

print(m2)

print(m3)

print(m4)

print(m5)

print(m6)

print(m7)

The chamfer distance between test_original.obj and test_only_move.obj is 6765.8349609375
The chamfer distance between test_original.obj and test_0.5.obj is 45.04317092895508
The chamfer distance between test_original.obj and test_2.obj is 184.780517578125
The chamfer distance between test_original.obj and test_ro1.obj is 240.19223022460938
The chamfer distance between test_original.obj and test_ro2_move.obj is 3924.61669921875
The chamfer distance between test_original.obj and test_sc_mv_ro.obj is 4707.7802734375
The chamfer distance between test_original.obj and 29_0.obj is 323.88909912109375


## Function: load obj -> point cloud  

[normalize the pt coordinate inside a a sphere of radius 1 centered at (0,0,0)  ]

In [36]:
def obj_to_cloud(filename):
  this_obj = os.path.join(filename)

  # load_obj
  verts, faces, aux = load_obj(this_obj)

  # transfer to tensor
  faces_idx = faces.verts_idx.to(device)
  verts = verts.to(device)

  #normalize
  center = verts.mean(0)
  verts = verts - center
  scale = max(verts.abs().max(0)[0])
  verts = verts / scale

  # construct to PyTorch3d data structure
  this_mesh = Meshes(verts=[verts], faces=[faces_idx])

  # sample pt from mesh
  this_point_cloud = sample_points_from_meshes(this_mesh, 5000)

  x, y, z = this_point_cloud.clone().detach().cpu().squeeze().unbind(1)    
  fig = plt.figure(figsize=(15, 15))
  ax = Axes3D(fig)
  ax.scatter3D(x, z, -y)
  ax.set_xlabel('x')
  ax.set_ylabel('z')
  ax.set_zlabel('y')
  ax.set_title('fig_{}.png'.format(filename+' normalized version'))
  ax.view_init(190, 30)
  plt.savefig('fig_{}.png'.format(filename+' normalized version'))
  plt.show()

  return this_point_cloud

In [37]:
def compute_loss(filename1, filename2):


  sample_trg = obj_to_cloud(filename1)
  sample_src = obj_to_cloud(filename2)
  
  loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    
  # and (b) the edge length of the predicted mesh
  #loss_edge = mesh_edge_loss(sample_src)
    
  # mesh normal consistency
  #loss_normal = mesh_normal_consistency(sample_src)
    
  # mesh laplacian smoothing
  #loss_laplacian = mesh_laplacian_smoothing(sample_src, method="uniform")
    
  # Weighted sum of the losses
  #loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian
  message = "The chamfer distance between "+filename1+" and "+filename2+" is "+str(loss_chamfer.item())

  return message

In [38]:
m1 = compute_loss('test_original.obj', 'test_only_move.obj')
m2 = compute_loss('test_original.obj', 'test_0.5.obj')
m3 = compute_loss('test_original.obj', 'test_2.obj')
m4 = compute_loss('test_original.obj', 'test_ro1.obj')
m5 = compute_loss('test_original.obj', 'test_ro2_move.obj')
m6 = compute_loss('test_original.obj', 'test_sc_mv_ro.obj')
m7 = compute_loss('test_original.obj', '29_0.obj')


/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_original.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_only_move.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

/usr/local/lib/python3.6/dist-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: test_sc_mv_ro.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
print(m1)

print(m2)

print(m3)

print(m4)

print(m5)

print(m6)

print(m7)

The chamfer distance between test_original.obj and test_only_move.obj is 0.0007834716234356165
The chamfer distance between test_original.obj and test_0.5.obj is 0.000807184842415154
The chamfer distance between test_original.obj and test_2.obj is 0.0007999422959983349
The chamfer distance between test_original.obj and test_ro1.obj is 0.27033132314682007
The chamfer distance between test_original.obj and test_ro2_move.obj is 0.2731636166572571
The chamfer distance between test_original.obj and test_sc_mv_ro.obj is 0.31229323148727417
The chamfer distance between test_original.obj and 29_0.obj is 0.25487494468688965
